# TVM 模块插件

```{topic} 导航
建议暴露能够修改由调用者传递的内存并调用运行时 API 进行内存分配的函数，允许插件模块创建并返回托管对象。然而，需要记住以下几个限制：

- 如果模块返回对象，需要确保在卸载模块之前销毁该对象。否则，由于调用未加载的析构函数，可能会导致段错误（segfault）。
- 如果模块返回 PackedFunc，那么需要确保 DLL 的 libc 和 TVM 运行时库匹配。否则，由于 `std::function` 的不兼容性，可能会导致段错误（segfault）。
```

::::{dropdown} C++ 源码：
```{literalinclude} src/plugin_module.cc
:language: C++
```
::::

编译：

```bash
make outputs/libs/libtvm_plugin_module.so
```

Python 端使用：

In [1]:
import os
from pathlib import Path
import tvm
from tvm import te

In [4]:
mod = tvm.runtime.load_module("outputs/libs/libtvm_plugin_module.so")

# 普通函数
assert mod["AddOne"](10) == 11
assert mod["SubOne"](10) == 9

高级用法——返回模块

In [5]:
mymod = mod["CreateMyModule"](10)
fadd = mymod["add"]
assert fadd(10) == 20
assert mymod["mul"](10) == 100